This notebook is meant to develop tools and plotting routines to analyse ball trajectories in fly pushing experiments

# Libraries

In [ ]:
import sys
from pathlib import Path
import matplotlib as mpl

mpl.rcParams["figure.figsize"] = (
    10,
    10,
)  # Change figure size including in the jupyter outputs.
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import mpmath
import cv2

sys.modules["sympy.mpmath"] = mpmath
from scipy import signal
import datetime
import dateutil
import bokeh.io
import holoviews as hv
from holoviews import opts

hv.extension(
    "bokeh",
    "matplotlib",
)
bokeh.io.output_notebook()

import panel as pn

sys.path.insert(0, "../../..")
sys.path.insert(0, "..")

from Utilities.Utils import *
from Utilities.Processing import *

import black
import jupyter_black

jupyter_black.load()

# Load dataframe

In [ ]:
VideoPath = Path(
    "/home/durrieu/mnt/labserver/labserver/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMazeBiS_15_Steel_Wax/Female_Starved_noWater/221116/102044_s0a0_p6-0/Arena4/Arena4.mp4"
)

data = pd.read_csv(
    VideoPath.parent.joinpath("BallPositions.csv").as_posix(),
)

In [ ]:
# Mac path

VideoPath = Path(
    "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMazeBiS_15_Steel_Wax/Female_Starved_noWater/221116/102044_s0a0_p6-0/Arena4/Arena4.mp4"
)

data = pd.read_csv(
    VideoPath.parent.joinpath("BallPositions.csv").as_posix(),
)

# Magnet events detection

## Manual detection

looking at the video, events can be annotated manually.

### Example of manual annotation

Gathered for arena 4 of very first example vid : '/home/durrieu/mnt/labserver/labserver/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMazeBiS_15_Steel_Wax/Female_Starved_noWater/221116/102044_s0a0_p6-0/Arena4/Arena4.mp4'

In [ ]:
Events = [
    [6, (4 * 60 + 25)],
    [(4 * 60 + 37), (4 * 60 + 59)],
    [(5 * 60 + 9), (5 * 60 + 28)],
    [(6 * 60), (7 * 60 + 5)],
    [(7 * 60 + 30), (9 * 60 + 14)],
    [(9 * 60 + 27), (12 * 60 + 8)],
    [(12 * 60 + 29), (14 * 60 + 8)],
    [(14 * 60 + 45), (16 * 60 + 32)],
    [(17 * 60 + 5), (18 * 60 + 30)],
    [(19 * 60 + 19), (20 * 60 + 46)],
    [(22 * 60 + 5), (23 * 60 + 5)],
    [(24 * 60 + 18), (25 * 60 + 17)],
    [(25 * 60 + 59), (26 * 60 + 29)],
    [(27 * 60 + 57), (29 * 60 + 23)],
    [(30 * 60 + 10), (31 * 60 + 42)],
    [(32 * 60 + 42), (34 * 60 + 41)],
    [(35 * 60 + 20), (36 * 60 + 53)],
    [(38 * 60), (40 * 60 + 11)],
    [(41 * 60 + 5), (42 * 60 + 17)],
    [(43 * 60 + 28), (45 * 60 + 34)],
    [(46 * 60 + 38), (49 * 60 + 21)],
    [(51 * 60 + 10), (53 * 60 + 7)],
    [(54 * 60 + 31), (58 * 60 + 51)],
]

Note that in this case values were already converted into seconds, but with frame2time function, they could have been entered as h:m:s too.

### Saving the time for further use

In [ ]:
# Convert all seconds in date format, easier to read for humans, and accepted by frame2time function.
Converted = [
    [str(datetime.timedelta(seconds=elements)) for elements in subs] for subs in Events
]

In [ ]:
MagnetEvents = np.array(Converted)

In [ ]:
## Save cropping parameters in a file for further use
if VideoPath.parent.joinpath("Magnet_Events.npy").exists() is True:
    choice = input("File already exists! Overwrite? [y/n]")

    if choice == "n":
        print("File unchanged.")

    elif choice == "y":
        np.save(VideoPath.parent.joinpath("Magnet_Events.npy").as_posix(), MagnetEvents)
        print("File updated.")

    else:
        print("invalid input")

else:
    np.save(VideoPath.parent.joinpath("Magnet_Events.npy").as_posix(), MagnetEvents)

## Automatic detection of magnet events

### Extract x and y from the dataframe

In [ ]:
RawTraj = data.loc[:, "ypos"]
frames = data.loc[:, "frame"]

### Apply smoothing

In [ ]:
x = frames.values
y = RawTraj.values

#### Savitzky–Golay filter

Worked well but I don't understand it so well so it required a lot of empirical tuning.

In [ ]:
# found around 20 events so also 20 pushes. according to https://www.sciencedirect.com/science/article/pii/S2211379718314761#f0005 , window length can be empircally tested or chosen as number of significant lobes / Datapoints

WinLen = round(len(y) / 500)  # 75 not bad, 100 even better
polyorder = 4

ysmooth = signal.savgol_filter(y, window_length=WinLen, polyorder=polyorder)

SavGo_Plot = hv.Curve(y) * hv.Curve(ysmooth).opts(tools=["hover"])

In [ ]:
#### Fourier transform

In [ ]:
ftrans = np.fft.rfft(y)
ysmooth = np.fft.irfft(ftrans)

TransfCurves = hv.Curve(y) * hv.Curve(ysmooth)
# TransfCurves

Note : Fourier's transform doesn't yield good results when there's not obvious frequency that can be discriminated from noise.

*Correction* : In my first implementation I tried to identify a frequently reoccuring frequency, which doesn't happen. Instead, I should use Fourier transform as a low pass filter to only keep biologically plausible frequencies.

#### Low pass filter

In [ ]:
# Filter requirements.
# fs = 80.0  # sample rate, Hz
# T = 1  # Sample Period
cutoff = 0.01  # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
# nyq = 0.5 * fs  # Nyquist Fre#quency
order = 2  # sin wave can be approx represented as quadratic
# n = int(T * fs)  # total number of samples

In [ ]:
def butter_lowpass_filter(
    data,
    cutoff,
    order,
    fs=None,
):
    # normal_cutoff = cutoff / nyq
    # Get the filter coefficients
    b, a = signal.butter(order, cutoff, btype="low", analog=False)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
# Filter parameters
cutoff = 0.01  # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
order = 1  # sin wave can be approx represented as quadratic

ysmooth = butter_lowpass_filter(y, cutoff, order)
TransfCurves = (
    hv.Curve(y).opts(
        height=1000,
        width=1000,
    )
    * hv.Curve(ysmooth)
)
TransfCurves

Note : Worked very well, did not see much impact of frequency and time period on the smoothing quality. Cutoff was the most critical.

> Also, scipy butter doesnt require all the frequency, time period stuff, it just requires a cutoff frequency + order, which makes it simpler to test empirically.

### Detect peaks on smoothed data

For now, low pass butter filter used.

In [ ]:
%matplotlib inline
DerY = np.diff(ysmooth)
plt.plot(DerY)

In [ ]:
Magnetpeaks = signal.find_peaks(
    DerY,
    distance=1000,
    height=1,
    # prominence=140,
    # width=2000,
    # threshold=-140,
    # plateau_size=np.arange(1,3000,1, dtype=int)
)

plt.plot(DerY)
plt.scatter(frames[Magnetpeaks[0]], DerY[Magnetpeaks[0]], color="orange")
plt.vlines(261428, ymin=-1, ymax=1, color="green", linestyles="dashed")

>Note: had to adapt peak detecion parameters to account for the change in filtering method.

In [ ]:
ManualStarts = [
    frame2time(events, fps=80, reverse=True, clockformat=True)
    for events in MagnetEvents[:, 0]
]
print(ManualStarts)
print(Magnetpeaks[0])

In [ ]:
# Adding manual events definition in order to draw them on the plots along with detections

ManualEnds = [
    frame2time(events, fps=80, reverse=True, clockformat=True)
    for events in MagnetEvents[:, 1]
]
ManualStarts = [
    frame2time(events, fps=80, reverse=True, clockformat=True)
    for events in MagnetEvents[:, 0]
]

In [ ]:
RevY = DerY * -1
FliesPeaks = signal.find_peaks(
    RevY,
    distance=2000,
    height=0.2,
    # prominence=140,
    # width=2000,
    # threshold=-140,
    # plateau_size=np.arange(1,3000,1, dtype=int)
)

# Old plotting routine using matplotlib
# plt.plot(RevY)
# plt.scatter(frames[FliesPeaks[0]], RevY[FliesPeaks[0]], color='orange')
# plt.vlines(frames[ManualEnds], ymin=-1, ymax=1, color='green', linestyles='dashed')

# Better plotting routine using holoviews
FlyPeaks_Plot = (
    hv.Curve(RevY).opts(
        height=1000,
        width=1000,
    )
    * hv.Points((frames[FliesPeaks[0]], RevY[FliesPeaks[0]])).opts(
        size=15,
        marker="x",
        color="red",
    )
    * hv.Spikes(ManualEnds)
)

# FlyPeaks_Plot

In [ ]:
print(f"manual recording: {frame2time(21200, fps=80, clockformat=True)}")
print(f"auto detected: {frame2time(19656, fps=80, clockformat=True)}")
frame2time((21200 - 19656), fps=80)

## Methods Comparison

In [ ]:
%matplotlib inline

ManualEnds = [
    frame2time(events, fps=80, reverse=True, clockformat=True)
    for events in MagnetEvents[:, 1]
]
ManualStarts = [
    frame2time(events, fps=80, reverse=True, clockformat=True)
    for events in MagnetEvents[:, 0]
]

AutoMag = Magnetpeaks[0]
AutoFly = FliesPeaks[0]


AccuracyCheck_Plot = plt.plot(DerY)
plt.scatter(frames[AutoMag], DerY[AutoMag], color="green", s=25, marker="x")
plt.scatter(frames[AutoFly], DerY[AutoFly], color="red", s=25, marker="x")
plt.vlines(frames[ManualEnds], ymin=-1, ymax=1, color="red", linestyles="dashed")
plt.vlines(frames[ManualStarts], ymin=-1, ymax=1, color="green", linestyles="dashed")
plt.figure(dpi=300)

print(
    f"Manually labeled values: {([frame2time(event, fps=80, clockformat=True) for event in ManualEnds])}"
)

print(
    f"Automatically labeled vvalues: {([frame2time(event, fps=80, clockformat=True) for event in AutoMag])}"
)

Note: up peaks are actually starts, which makes sense, as y is reverted. ends seem also detectable from the same method, because flies final pushes are usually quick enough for the derivative to be significantly different from the rest

Note also that another strategy would be only to detect magnet events and work from there (magnet events are new starts, e.g. previous trial should have ended shortly before).

## Test on using only magnet events for trial splitting

### Plot time between end of previous trial and magnet even distribution

In [ ]:
Starts = [
    frame2time(events, fps=80, reverse=True, clockformat=True)
    for events in MagnetEvents[:, 0]
]

Ends = [
    frame2time(events, fps=80, reverse=True, clockformat=True)
    for events in MagnetEvents[:, 1]
]

In [ ]:
latency = [(start - end) for start, end in zip(Starts[1:], Ends[:-1])]

Latency_Hist = plt.hist(latency)

In [ ]:
max(latency) / 80

Note: Might be too spread. At max, there is still almost 2 minutes latency.

In [ ]:
average_latency = np.average(latency)

EstimatedEnds = [(i - average_latency) for i in AutoMag[1:]]

AvgLat_Plot = (
    hv.Curve(DerY).opts(
        height=1000,
        width=1000,
    )
    * hv.Points((frames[AutoMag], DerY[AutoMag])).opts(
        size=15,
        marker="x",
        color="red",
    )
    * hv.Spikes(ManualEnds)
    * hv.Spikes(EstimatedEnds).opts(
        color="red",
        spike_length=1,
        position=-0.5,
    )
)

# AvgLat_Plot

In [ ]:
# Biggest discrepancy:
frame2time(241461 - 236880, fps=80, clockformat=True)

# Timestamps:
print(f"manual recording: {frame2time(236880, fps=80, clockformat=True)}")
print(f"auto detected: {frame2time(241461, fps=80, clockformat=True)}")

### Test timestamps accuracy

In [ ]:
# Don't know what I was trying to do here
# cap = cv2.VideoCapture(VideoPath.as_posix())
# cap.set(cv2.CAP_PROP_FRAMES, AutoMag[0] - 1)
# res, frame = cap.read()

In [ ]:
frame2time(AutoMag[0], fps=80, clockformat=True)

## 2-steps trials splits

Here the idea is to use magnet events as a first splitting method as these are super accurate. Then, remove some initial value to avoid early weirdness due to magnet still being around. Finally, within each detected trial, set a threshold and stop the trial as soon a the threshold had been reached for the first time.

In [ ]:
# Add a last value used as token to implement last trial
data = pd.read_csv(
    VideoPath.parent.joinpath("BallPositions.csv").as_posix(),
)

# Add smoothed values to the dataframe

data["ysmooth"] = ysmooth
# MagnetEvents = np.append(AutoMag, max(data.frame))

data

In [ ]:
len(MagnetEvents)

In [ ]:
Magnetpeaks

In [ ]:
MagnetEvents = np.append(0, Magnetpeaks[0])

In [ ]:
data = pd.read_csv(
    VideoPath.parent.joinpath("BallPositions.csv").as_posix(),
)
data["ysmooth"] = ysmooth
data["TrialNumber_init"] = None

for t in range(len(MagnetEvents) - 1):

    data.loc[MagnetEvents[t] : MagnetEvents[t + 1], "TrialNumber_init"] = t + 1

data.loc[MagnetEvents[-1] :, "TrialNumber_init"] = len(MagnetEvents)


data

In [ ]:
data = data.dropna(subset="TrialNumber_init")

data.head()

In [ ]:
GroupDf = data.groupby("TrialNumber_init", as_index=False)

data = GroupDf.apply(lambda x: x.reset_index(drop=True)).reset_index()

data.rename(columns={"level_1": "Time"}, inplace=True)
data = data.sort_values(by=["TrialNumber_init", "Time"])

In [ ]:
Curves = (
    hv.Curve(
        data=data,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber_init",
        ],
    )
    .groupby("TrialNumber_init")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Viridis"),
        tools=[
            "hover",
            "crosshair",
        ],
        muted=True,
    )
    .overlay()
)
Curves

Trial 14 is a false positive. Upon checking the video it appears the magnet has been accidentally moved and replaced. 

In [ ]:
# Drop the values associated with Trial 14
data = data[data["TrialNumber_init"] != 14]
# Also drop the associated Magnet Event, which is the 13th one
MagnetEvents = np.delete(MagnetEvents, 13)

> I keep alternatineg between detecting an outlier and not detecting it. Weird.

This is just an example of pandas dataframe browsing. It's not used in the code.

In [ ]:
data.loc[(data["TrialNumber_init"] == 23) & (data["Time"] == 15937)]

In [ ]:
frame2time(261428, fps=80, clockformat=True)

After a first glance at the data, two things need to be done.

### Find 'success' threshold

In [ ]:
# First looking for smallest max value : (using min because reverted axis)
data.groupby("TrialNumber_init").min()

smallest max ypos is 38. Round it to 40 and keep only values that are above this.

In [ ]:
data.loc[data["ypos"] <= 40].groupby("TrialNumber_init").first()

In [ ]:
Thresh_Ends = data.loc[data["ypos"] <= 40].groupby("TrialNumber_init").first()["frame"]
Thresh_Ends

In [ ]:
data["TrialNumber"] = None  # Need reset otherwise no NA drops

for t in range(len(MagnetEvents)):

    data.loc[MagnetEvents[t] : Thresh_Ends.values[t], "TrialNumber"] = t + 1

In [ ]:
data = data.dropna(subset="TrialNumber")

data = data.drop(["level_0", "Unnamed: 0"], axis=1)

data

### Remove first 500 frames

In [ ]:
# Drop positions before 500 frames
data.drop(data.loc[data["Time"] <= 500].index, inplace=True)

# Reset Time
# data = GroupDf.apply(lambda x: x.reset_index(drop=True)).reset_index()
# data.rename(columns={'level_1': 'CorrTime'}, inplace=True)
# data = data.sort_values(by=['TrialNumber', 'CorrTime'])

In [ ]:
# Replot to check data quality
Curves = (
    hv.Curve(
        data=data,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber",
        ],
    )
    .groupby("TrialNumber")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Plasma"),
        tools=[
            "hover",
        ],
        muted=True,
    )
    .overlay()
)

Curves

## Save HTML plot

In [ ]:
hv.save(Curves, "/mnt/labserver/DURRIEU_Matthias/Pictures/Arena4.html", backend="bokeh")

# Plotting ball position per trials

## Load Magnet events

In [ ]:
MagnetEvents = np.load(VideoPath.parent.joinpath('Magnet_Events.npy').as_posix(),
                 allow_pickle = True)
MagnetEvents

In [ ]:
Starts = [frame2time(events, fps=80, reverse=True, clockformat=True) for events in MagnetEvents[:, 0]]

Ends = [frame2time(events, fps=80, reverse=True, clockformat=True) for events in MagnetEvents[:, 1]]

## Map Trials to Dataframe

In [ ]:
data = pd.read_csv(VideoPath.parent.joinpath('BallPositions.csv').as_posix(),)

data['TrialNumber'] = None

for t in range(len(Events)):
    data.loc[Starts[t]:Ends[t], 'TrialNumber'] = t+1

data = data.dropna(subset='TrialNumber')

## Add timer for each trial

In [ ]:
GroupDf = data.groupby('TrialNumber', as_index=False)

data = GroupDf.apply(lambda x: x.reset_index(drop=True)).reset_index()

data.rename(columns = {'level_1':'Time'}, inplace = True)
data = data.sort_values(by = ['TrialNumber', 'Time'])

## Plot all trials ball positions over time

In [ ]:
Curves = hv.Curve(data= data,
                  kdims=['Time'],
                  vdims=['ypos',
                         'TrialNumber',
                         ],
                  ).groupby('TrialNumber'
                            ).opts(
    height = 1000,
    width = 1000,
    invert_yaxis = True,
    color = hv.Palette('Viridis'),
    tools = ['hover',
             'crosshair',
             ],
).overlay()
Curves

## Compute and plot trial durations as function of trialNumber

In [ ]:
Trialduration = GroupDf.size()

In [ ]:
vals = Trialduration.values

Trials = vals[:, 0]
Durations = vals[:, 1]

In [ ]:
DurationPlot = hv.Points((Trials, Durations)).opts(
    size = 15,
    marker = '+',
    color = 'red',
)

DurationPlot

## Test with auto magnets

In [ ]:
data = pd.read_csv(VideoPath.parent.joinpath('BallPositions.csv').as_posix(),)

data['TrialNumber'] = None

for t in range(len(AutoMag)):
    if t == 0:
        data.loc[0:AutoMag[t], 'TrialNumber'] = t+1
    else:
        data.loc[AutoMag[t-1]+1:AutoMag[t], 'TrialNumber'] = t+1

data = data.dropna(subset='TrialNumber')

In [ ]:
GroupDf = data.groupby('TrialNumber', as_index=False)

data = GroupDf.apply(lambda x: x.reset_index(drop=True)).reset_index()

data.rename(columns = {'level_1':'Time'}, inplace = True)
data = data.sort_values(by = ['TrialNumber', 'Time'])

In [ ]:
Curves = hv.Curve(data= data,
                  kdims=['Time'],
                  vdims=['ypos',
                         'TrialNumber',
                         ],
                  ).groupby('TrialNumber'
                            ).opts(
    height = 1000,
    width = 1000,
    invert_yaxis = True,
    color = hv.Palette('Viridis'),
    tools = ['hover',
             'crosshair',
             ],
).overlay()
Curves

Note:

## Add estimated ends to avoid drop at the end of trials

In [ ]:
data = pd.read_csv(VideoPath.parent.joinpath('BallPositions.csv').as_posix(),)

data['TrialNumber'] = None

for t in range(len(AutoMag)-1):
    if t == 0:
        data.loc[0:EstimatedEnds[t], 'TrialNumber'] = t+1
    else:
        data.loc[AutoMag[t-1]+1:EstimatedEnds[t], 'TrialNumber'] = t+1

data = data.dropna(subset='TrialNumber')

In [ ]:
GroupDf = data.groupby('TrialNumber', as_index=False)

data = GroupDf.apply(lambda x: x.reset_index(drop=True)).reset_index()

data.rename(columns = {'level_1':'Time'}, inplace = True)
data = data.sort_values(by = ['TrialNumber', 'Time'])

In [ ]:
Curves = hv.Curve(data= data,
                  kdims=['Time'],
                  vdims=['ypos',
                         'TrialNumber',
                         ],
                  ).groupby('TrialNumber'
                            ).opts(
    height = 1000,
    width = 1000,
    invert_yaxis = True,
    color = hv.Palette('Viridis'),
    tools = ['hover',
             'crosshair',
             ],
).overlay()
Curves

Not accurate at all.

# Test pipeline with another arena

In [ ]:
# Linux path
VideoPath = Path(
    "/home/durrieu/mnt/labserver/labserver/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMazeBiS_15_Steel_Wax/Female_Starved_noWater/221116/102044_s0a0_p6-0/Arena5/Arena5.mp4"
)

In [ ]:
# Mac path for alternative data

VideoPath = Path(
    "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMazeBiS_15_Steel_Wax/Female_Starved_noWater/221116/102044_s0a0_p6-0/Arena5/Arena5.mp4"
)


In [ ]:
data = pd.read_csv(
    VideoPath.parent.joinpath("BallPositions.csv").as_posix(),
)

data.head()

In [ ]:
x = data.loc[:, "frame"].values
y = data.loc[:, "ypos"].values

In [ ]:
# Filter parameters
cutoff = 0.01  # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
order = 1  # sin wave can be approx represented as quadratic

ysmooth = butter_lowpass_filter(y, cutoff, order)
TransfCurves = (
    hv.Curve(y).opts(
        height=1000,
        width=1000,
    )
    * hv.Curve(ysmooth)
)
TransfCurves

In [ ]:
%matplotlib inline
DerY = np.diff(ysmooth)
plt.plot(DerY)

In [ ]:
Magnetpeaks = signal.find_peaks(
    DerY,
    distance=1000,
    height=1,
    # prominence=140,
    # width=2000,
    # threshold=-140,
    # plateau_size=np.arange(1,3000,1, dtype=int)
)

plt.plot(DerY)
plt.scatter(x[Magnetpeaks[0]], DerY[Magnetpeaks[0]], color="orange")

Here the very begining of the experiment seems to be a magnet event, so no need to add 0 to the list of magnet events. However, I see another magnet event at the very end of the video, which is not relevant as the experiment is over.

In [ ]:
data["ysmooth"] = ysmooth
MagnetEvents = Magnetpeaks[0][:-1]

In [ ]:
print(MagnetEvents[-1])
print(Magnetpeaks[0][-1])

In [ ]:
data["TrialNumber_init"] = None

for t in range(len(MagnetEvents) - 1):

    data.loc[MagnetEvents[t] : MagnetEvents[t + 1], "TrialNumber_init"] = t + 1

data.loc[MagnetEvents[-1] :, "TrialNumber_init"] = len(MagnetEvents)


data

In [ ]:
data = data.dropna(subset="TrialNumber_init")

data.head()

In [ ]:
GroupDf = data.groupby("TrialNumber_init", as_index=False)

data = GroupDf.apply(lambda x: x.reset_index(drop=True)).reset_index()

data.rename(columns={"level_1": "Time"}, inplace=True)
data = data.sort_values(by=["TrialNumber_init", "Time"])

In [ ]:
Curves = (
    hv.Curve(
        data=data,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber_init",
        ],
    )
    .groupby("TrialNumber_init")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Plasma"),
        tools=[
            "hover",
            "crosshair",
        ],
        muted=True,
    )
    .overlay()
)
Curves

> Trial 2 is incomplete, along with 10 and 11. I should remove them from the analysis.

## Incomplete trials removal

In some trials, flies don't push the ball all the way before it is replaced, let's implement a way to detect and remove these trials.

### Find minimum y position for each trial

Here I use the same method as for the success threshold, but I also set a cutoff value to identify any trial that has a minimum y position below this value.

In [ ]:
ymins = data.groupby("TrialNumber_init").min()["ypos"]

ymins

In [ ]:
print(f"mean: {round(np.average(ymins))}")
print(f"mean rounded to the nearest 10: {int(round(np.average(ymins),-1))}")
print(f"median: {np.median(ymins)}")

>Mean rounded to the nearest 10 seems to work well to identify threshold value. Could also go for the mean without rounding (or rounding to the nearest 5?).

In [ ]:
BadTrials = ymins.loc[ymins > 53].index.to_list()

In [ ]:
# Drop the values associated with detected bad trials
trimmed_data = data.loc[~data["TrialNumber_init"].isin(BadTrials)]

In [ ]:
# Also drop the associated Magnet Events
Mag_ymins = [v - 1 for v in BadTrials]
MagnetEvents = np.delete(MagnetEvents, Mag_ymins)

Then I can reuse the same threshold defined earlier as success threshold.

In [ ]:
trimmed_data.loc[trimmed_data["ypos"] <= 53].groupby("TrialNumber_init").first()

In [ ]:
Thresh_Ends = (
    trimmed_data.loc[trimmed_data["ypos"] <= 53]
    .groupby("TrialNumber_init")
    .first()["frame"]
)
Thresh_Ends

In [ ]:
trimmed_data["TrialNumber"] = None  # Need reset otherwise no NA drops

for t in range(len(MagnetEvents)):

    trimmed_data.loc[MagnetEvents[t] : Thresh_Ends.values[t], "TrialNumber"] = t + 1

In [ ]:
trimmed_data = trimmed_data.dropna(subset="TrialNumber")

trimmed_data

In [ ]:
# Drop positions before 500 frames
trimmed_data.drop(trimmed_data.loc[trimmed_data["Time"] <= 500].index, inplace=True)

In [ ]:
Curves = (
    hv.Curve(
        data=trimmed_data,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber",
        ],
    )
    .groupby("TrialNumber")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Plasma"),
        tools=[
            "hover",
        ],
        muted=True,
    )
    .overlay()
)

Curves

In [ ]:
hv.save(Curves, "/mnt/labserver/DURRIEU_Matthias/Pictures/Arena5.html", backend="bokeh")

> This worked pretty well. Now we might want to save 1) The processed dataset 2) The plots

## Saving useful data

### Saving processed data

> To do : drop useless columns and try saving as feather file.

In [ ]:
# drop level_0, index and TrialNumber_init columns from the trimmed_data dataframe
trimmed_data = trimmed_data.drop(columns=["level_0", "TrialNumber_init"]).reset_index()

In [ ]:
trimmed_data.to_feather(VideoPath.parent.joinpath("BallPositions_Processed.feather"))

In [ ]:
# save the processed dataset as npy file in same folder as video

checksave(
    path=VideoPath.parent.joinpath("BallPositions_Processed.feather"),
    object="dataframe",
    file=trimmed_data,
)

Just as test, see if I can recover the feather file properly

In [ ]:
imp_data = pd.read_feather(VideoPath.parent.joinpath("BallPositions_Processed.feather"))

In [ ]:
GroupDf = trimmed_data.groupby("TrialNumber", as_index=False)

Trialduration = GroupDf.size()

vals = Trialduration.values

Trials = vals[:, 0]
Durations = vals[:, 1]

DurationPlot = hv.Points((Trials, Durations)).opts(
    size=15,
    marker="+",
    color="red",
)

DurationPlot

# Apply the pipeline to the third arena with new methods

In [ ]:
# Linux path
VideoPath = Path(
    "/mnt/labserver/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMazeBiS_15_Steel_Wax/Female_Starved_noWater/221116/102044_s0a0_p6-0/Arena6/Arena6.mp4"
)

In [ ]:
# Mac path for alternative data

VideoPath = Path(
    "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMazeBiS_15_Steel_Wax/Female_Starved_noWater/221116/102044_s0a0_p6-0/Arena6/Arena6.mp4"
)

In [ ]:
data = pd.read_csv(
    VideoPath.parent.joinpath("BallPositions.csv").as_posix(),
)

x = data.loc[:, "frame"].values
y = data.loc[:, "ypos"].values

cutoff = 0.01  # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
order = 1  # sin wave can be approx represented as quadratic

ysmooth = butter_lowpass_filter(y, cutoff, order)
TransfCurves = hv.Curve(y).opts(
    height=1000,
    width=1000,
) * hv.Curve(ysmooth)

DerY = np.diff(ysmooth)

Magnetpeaks = signal.find_peaks(
    DerY,
    distance=1000,
    height=1,
    # prominence=140,
    # width=2000,
    # threshold=-140,
    # plateau_size=np.arange(1,3000,1, dtype=int)
)

plt.plot(DerY)
plt.scatter(x[Magnetpeaks[0]], DerY[Magnetpeaks[0]], color="orange")

data["ysmooth"] = ysmooth
MagnetEvents = Magnetpeaks[0][:-1]

> In this example, I see that there are small events that could be magnet, but the first one is quite late. Let's check this out.

In [ ]:
frame2time(MagnetEvents[0], fps=80, clockformat=True)

Definitely a magnet event. What about the the smaller ones?

In [ ]:
# find the indices of DerY where DerY value is above 0.5
Magnetpeaks_Alt = signal.find_peaks(
    DerY,
    distance=1000,
    height=0.5,
    # prominence=140,
    # width=2000,
    # threshold=-140,
    # plateau_size=np.arange(1,3000,1, dtype=int)
)

plt.plot(DerY)
plt.scatter(x[Magnetpeaks_Alt[0]], DerY[Magnetpeaks_Alt[0]], color="orange")

In [ ]:
frame2time(Magnetpeaks_Alt[0][1], fps=80, clockformat=True)

After checking, I find that these smaller peaks are magnet events that occur before the fly can manage to push the ball. These are thus called 'partial magnet events'.

There are to ways to look at this :
1. Replacing the magnet reset de facto the experiment. Thus, the trial is not valid and should be removed.
2. Flies are still experiencing the ball and potentially learning something. Therefore, removing these pseudo-trials might create a bias, especially since the first trial is the most important one.

## Detect and remove incomplete trials

Strategy: there seems to be a clear cutoff between complete and incomplete trials. Might need to adjust the threshold value from one experiment to the other but within one experiment, if settings are stable, peak detection should remain the same.

In [ ]:
# Find elements that are in Magnetpeaks_Alt[0] but not in Magnetpeaks[0]
np.setdiff1d(Magnetpeaks_Alt[0], Magnetpeaks[0])

> Note: Here from trial to trial what can change is whether there is a magnet event at the very begining, the very end or neither. If there is no trial at the begining, I need to add a 0 to the list of magnet events. If there is no trial at the end, I need to remove the last magnet event from the list. 

In this dataset, there is a magnet event close to the end. How to handle that? Perhaps no need to, as either it will be a valid trial and taken into consideration, or it won't and be automatically removed following method developped on the previous dataset. Why not try the same for early magnet events and systematically add a 0 value to all Magnet events arrays?

In [ ]:
print(Magnetpeaks[0])
print(len(DerY))

In [ ]:
data["ysmooth"] = ysmooth
MagnetEvents = np.append(0, Magnetpeaks_Alt[0])

data["TrialNumber_init"] = None

for t in range(len(MagnetEvents) - 1):
    data.loc[MagnetEvents[t] : MagnetEvents[t + 1], "TrialNumber_init"] = t + 1

data.loc[MagnetEvents[-1] :, "TrialNumber_init"] = len(MagnetEvents)

GroupDf = data.groupby("TrialNumber_init", as_index=False)

data = GroupDf.apply(lambda x: x.reset_index(drop=True)).reset_index()

data.rename(columns={"level_1": "Time"}, inplace=True)
data = data.sort_values(by=["TrialNumber_init", "Time"])

Curves = (
    hv.Curve(
        data=data,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber_init",
        ],
    )
    .groupby("TrialNumber_init")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Plasma"),
        tools=[
            "hover",
            "crosshair",
        ],
        muted=True,
    )
    .overlay()
)
Curves

In [ ]:
Curves = (
    hv.Curve(
        data=data,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber_init",
        ],
    )
    .groupby("TrialNumber_init")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Plasma"),
        tools=[
            "hover",
            "crosshair",
        ],
        muted=True,
    )
    .overlay()
)

Curves

This is a very good example to test trial selection based on success threshold

In [ ]:
ymins = data.groupby("TrialNumber_init").min()["ypos"]

ymins

In [ ]:
print(f"mean: {round(np.average(ymins))}")
print(f"mean rounded to the nearest 10: {int(round(np.average(ymins),-1))}")
print(f"median: {np.median(ymins)}")

> Here getting the average correctly detects outliers, which are two first trials (partial magnet events) and the last one (incomplete trial at the end of the recording). However, the threshold is quite high and would remove a lot of data if applied to the whole dataset. Instead I should just take the highest value of the non-outliers and use it as a threshold.

In [ ]:
# Find the maximum value of ymins among values that are lower than the mean
succ_thresh = ymins[ymins < np.average(ymins)].max()

In [ ]:
BadTrials = ymins.loc[ymins > succ_thresh].index.to_list()

In [ ]:
trimmed_data = data.loc[~data["TrialNumber_init"].isin(BadTrials)]
Mag_ymins = [v - 1 for v in BadTrials]
MagnetEvents = np.delete(MagnetEvents, Mag_ymins)


Thresh_Ends = (
    trimmed_data.loc[trimmed_data["ypos"] <= 53]
    .groupby("TrialNumber_init")
    .first()["frame"]
)

trimmed_data["TrialNumber"] = None  # Need reset otherwise no NA drops

for t in range(len(MagnetEvents)):
    trimmed_data.loc[MagnetEvents[t] : Thresh_Ends.values[t], "TrialNumber"] = t + 1

trimmed_data = trimmed_data.dropna(subset="TrialNumber")

trimmed_data

### Convert Time in frames in time in seconds

In [ ]:
trimmed_data["Time_sec"] = trimmed_data["Time"] / 80

In [ ]:
# Drop positions before 500 frames
trimmed_data.drop(trimmed_data.loc[trimmed_data["Time"] <= 500].index, inplace=True)

Curves = (
    hv.Curve(
        data=trimmed_data,
        kdims=["Time_sec"],
        vdims=[
            "ysmooth",
            "TrialNumber",
        ],
    )
    .groupby("TrialNumber")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Viridis"),
        tools=[
            "hover",
        ],
        xlabel="Time (sec)",
        ylabel="Ball position (y coordinates)",
        fontscale=2,
        # muted=True,
    )
    .overlay()
)

Curves

In [ ]:
hv.save(
    Curves, "/mnt/labserver/DURRIEU_Matthias/Pictures/FyssenReport/SignatureCurve.png"
)
hv.save(
    Curves, "/mnt/labserver/DURRIEU_Matthias/Pictures/FyssenReport/SignatureCurve.html"
)

In [ ]:
import holoviews as hv
from bokeh.io import export_svgs

p = hv.render(Curves, backend="bokeh")
p.output_backend = "svg"
export_svgs(
    p,
    filename="/mnt/labserver/DURRIEU_Matthias/Pictures/FlyTrajectories3_betterlabels.svg",
)

What I note here is that removing the partial magnet events removed a lot of information from the first trial and I end up with a very short first trialn which is not what actually happens. I should thus keep the partial magnet events and just remove the incomplete trials at the end.

## Same procedure but keeping partial magnet events

In [ ]:
data = pd.read_csv(
    VideoPath.parent.joinpath("BallPositions.csv").as_posix(),
)

data["ysmooth"] = ysmooth
MagnetEvents = np.append(0, Magnetpeaks[0])

data["TrialNumber_init"] = None

for t in range(len(MagnetEvents) - 1):

    data.loc[MagnetEvents[t] : MagnetEvents[t + 1], "TrialNumber_init"] = t + 1

data.loc[MagnetEvents[-1] :, "TrialNumber_init"] = len(MagnetEvents)

GroupDf = data.groupby("TrialNumber_init", as_index=False)

data = GroupDf.apply(lambda x: x.reset_index(drop=True)).reset_index()

data.rename(columns={"level_1": "Time"}, inplace=True)
data = data.sort_values(by=["TrialNumber_init", "Time"])

ymins = data.groupby("TrialNumber_init").min()["ypos"]

ymins

In [ ]:
print(f"mean: {round(np.average(ymins))}")

Now my outlier detection method is too coarse. Not enough outliers so the average value is lower and detects trial 1 as outlier which it's not. Let's use a more precise method.

### Find outliers in ymins

#### Method 1: Z-score

In [ ]:
# find outliers in ymins
from scipy import stats

np.abs(stats.zscore(ymins))

Here I can clearly see where the outlier is but I need to manually set a threshold, which is no ideal.

In [ ]:
interquart = np.percentile(ymins, 75) - np.percentile(ymins, 25)

upper = ymins >= (np.percentile(ymins, 75) + 1.5 * interquart)

upper

In [ ]:
hv.BoxWhisker(ymins)

Here I also set 1st trial as outlier.

In [ ]:
rg = np.random.default_rng()


def draw_bs_rep(data, func, rg):
    """Compute a bootstrap replicate from data."""
    bs_sample = rg.choice(data, size=len(data))
    return func(bs_sample)


def draw_bs_ci(data, func=np.mean, rg=rg, n_reps=2000):
    """Sample bootstrap multiple times and compute confidence interval"""
    bs_reps = np.array([draw_bs_rep(data, func, rg) for _ in range(n_reps)])
    conf_int = np.percentile(bs_reps, [2.5, 97.5])
    return conf_int

In [ ]:
ymins_CI = draw_bs_ci(ymins)

Looks good!

In [ ]:
succ_thresh = ymins[ymins < ymins_CI[1]].max()

BadTrials = ymins.loc[ymins > succ_thresh].index.to_list()

In [ ]:
trimmed_data = data.loc[~data["TrialNumber_init"].isin(BadTrials)]
Mag_ymins = [v - 1 for v in BadTrials]
MagnetEvents = np.delete(MagnetEvents, Mag_ymins)


Thresh_Ends = (
    trimmed_data.loc[trimmed_data["ypos"] <= 53]
    .groupby("TrialNumber_init")
    .first()["frame"]
)

trimmed_data["TrialNumber"] = None  # Need reset otherwise no NA drops

for t in range(len(MagnetEvents)):

    trimmed_data.loc[MagnetEvents[t] : Thresh_Ends.values[t], "TrialNumber"] = t + 1

trimmed_data = trimmed_data.dropna(subset="TrialNumber")

trimmed_data

In [ ]:
# Drop positions before 500 frames
trimmed_data.drop(trimmed_data.loc[trimmed_data["Time"] <= 500].index, inplace=True)

Curves = (
    hv.Curve(
        data=trimmed_data,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber",
        ],
    )
    .groupby("TrialNumber")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Plasma"),
        tools=[
            "hover",
        ],
        muted=True,
    )
    .overlay()
)

Curves

# Apply pipeline to an arena from a different video

Here we test the generalisability of the pipeline to a different arena coming from a different video. We expect potential parameter tuning to be required (e.g. for peak detection).

In [ ]:
# Load the data
DataPath = Path(
    "/mnt/labserver/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMaze_15stepped_gated_bowtie/Starved_noWater/230209/111026_s0a0_p0-0/Arena6/BallPositions.csv"
)

data = pd.read_csv(DataPath.as_posix())

In [ ]:
%matplotlib inline

x = data.loc[:, "frame"].values
y = data.loc[:, "ypos"].values

cutoff = 0.01  # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
order = 1  # sin wave can be approx represented as quadratic

ysmooth = butter_lowpass_filter(y, cutoff, order)
TransfCurves = (
    hv.Curve(y).opts(
        height=1000,
        width=1000,
    )
    * hv.Curve(ysmooth)
)

DerY = np.diff(ysmooth)

Magnetpeaks = signal.find_peaks(
    DerY,
    distance=1000,
    height=0.7,
    # prominence=140,
    # width=2000,
    # threshold=-140,
    # plateau_size=np.arange(1,3000,1, dtype=int)
)

data["ysmooth"] = ysmooth

plt.plot(DerY)
plt.scatter(x[Magnetpeaks[0]], DerY[Magnetpeaks[0]], color="orange")
plt.gca().invert_yaxis()
# plt.show()

plot smooth trajectories

In [ ]:
%matplotlib inline

x = data.loc[:, "frame"].values
y = data.loc[:, "ypos"].values

cutoff = 0.01  # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
order = 1  # sin wave can be approx represented as quadratic

ysmooth = butter_lowpass_filter(y, cutoff, order)
TransfCurves = hv.Curve(y) * hv.Curve(ysmooth).opts(
    height=1000,
    width=1000,
    line_width=2,
)
TransfCurves

In new video settings, peak detection threshold from older videos is too low.

In [ ]:
idx = 0
for i in Magnetpeaks[0]:
    print(idx)
    frame2time(i, 80, clockformat=True)
    idx += 1

In [ ]:
# with open(DataPath.parent.joinpath("note.txt").as_posix(), "w") as f:
#     f.write("Bad tracking around 40 min. Redo tracking with better parameters.")

In [ ]:
# Select only the values actually corresponding to magnet events

In [ ]:
MagnetEvs_chk = [Magnetpeaks[0][i] for i in [17, ]]
idx = 0
for i in MagnetEvs_chk:
    print(idx)
    frame2time(i, 80, clockformat=True)
    idx += 1

In some cases, very rare, a magnet event is not detected on derivative, because the magnet was not replaced fast enough. In this case we can add a value to the list of magnet events.

In [ ]:
# Compute frame number for the manually selected magnet event
# 00:00:31
# 00:20:43
frame2time((20 * 60 + 43), 80, reverse=True)

In [ ]:
# Add this index to the list of magnet events
# add 139040 at the 3 position in MagnetEvs_chk
MagnetEvs_chk.insert(1, 99440)

for i in MagnetEvs_chk:
    frame2time(i, 80, clockformat=True)

In [ ]:
MagnetEvents = np.append(0, MagnetEvs_chk)

data["TrialNumber_init"] = None

for t in range(len(MagnetEvents) - 1):

    data.loc[MagnetEvents[t] : MagnetEvents[t + 1], "TrialNumber_init"] = t + 1

data.loc[MagnetEvents[-1] :, "TrialNumber_init"] = len(MagnetEvents)

GroupDf = data.groupby("TrialNumber_init", as_index=False)

data = GroupDf.apply(lambda x: x.reset_index(drop=True)).reset_index()

data.rename(columns={"level_1": "Time"}, inplace=True)
data = data.sort_values(by=["TrialNumber_init", "Time"])

In [ ]:
Curves = (
    hv.Curve(
        data=data,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber_init",
        ],
    )
    .groupby("TrialNumber_init")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Viridis"),
        tools=[
            "hover",
            "crosshair",
        ],
        muted=True,
    )
    .overlay()
)
Curves

Here Trial 8 should be discarded as it is simply the end of the video.

In [ ]:
# drop trials 14 and 15 and 16 + last.
# data.drop(data.loc[data["TrialNumber_init"] == 14].index, inplace=True)
# data.drop(data.loc[data["TrialNumber_init"] == 15].index, inplace=True)
# data.drop(data.loc[data["TrialNumber_init"] == 16].index, inplace=True)


# MagnetEvents = np.delete(MagnetEvents, 16 - 1)
# MagnetEvents = np.delete(MagnetEvents, 15 - 1)
# MagnetEvents = np.delete(MagnetEvents, 14 - 1)

In [ ]:
# Find highest value of TrialNumber_init
#data["TrialNumber_init"].max()

In [ ]:
# Remove data from last trial when too short
data.drop(
    data.loc[data["TrialNumber_init"] == data["TrialNumber_init"].max()].index,
    inplace=True,
)

# Also remove last magnet event
MagnetEvents = np.delete(MagnetEvents, -1)

In [ ]:
idx = 0
for i in MagnetEvents:
    print(idx)
    frame2time(i, 80, clockformat=True)
    idx += 1

In [ ]:
#MagnetEvents = np.delete(MagnetEvents, 17)

In [ ]:
# Also drop the last element of MagnetEvents
#MagnetEvents = np.delete(MagnetEvents, data["TrialNumber_init"].max() - 1)

> This is a special case where the ball was initially not found in the video, resulting in NaNs that have to be removed.

In [ ]:
ymins = data.groupby("TrialNumber_init").min()["ysmooth"]

In [ ]:
succ_thresh = ymins.max()

In [ ]:
trimmed_data = data
# Mag_ymins = [v - 1 for v in BadTrials]
# MagnetEvents = np.delete(MagnetEvents, Mag_ymins)
trimmed_data.drop(trimmed_data.loc[trimmed_data["Time"] <= 500].index, inplace=True)

Thresh_Ends = (
    trimmed_data.loc[trimmed_data["ysmooth"] <= succ_thresh]
    .groupby("TrialNumber_init")
    .first()["frame"]
)

In [ ]:
trimmed_data["TrialNumber"] = None  # Need reset otherwise no NA drops

for t in range(len(MagnetEvents)):

    trimmed_data.loc[MagnetEvents[t] : Thresh_Ends.values[t], "TrialNumber"] = t + 1

In [ ]:
trimmed_data = trimmed_data.dropna(subset="TrialNumber")


Curves = (
    hv.Curve(
        data=trimmed_data,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber",
        ],
    )
    .groupby("TrialNumber")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Viridis"),
        tools=[
            "hover",
        ],
        muted=True,
    )
    .overlay()
)

Curves

Save data and plot

In [ ]:
# drop level_0, index and TrialNumber_init columns from the trimmed_data dataframe
trimmed_data = trimmed_data.drop(columns=["level_0", "TrialNumber_init"]).reset_index()

# save the processed dataset as feather file in same folder as video

checksave(
    path=DataPath.parent.joinpath("BallPositions_Processed.feather"),
    object="dataframe",
    file=trimmed_data,
)

In [ ]:
hv.save(Curves, DataPath.parent.joinpath("BallPositions_Processed.html"), fmt="html")

In [ ]:
add_note(
    DataPath,
    "Only one trial. Discarded as it would bias the analysis.",
)

Replot with different colors

In [ ]:
reload = pd.read_feather(
    "/Volumes/Transfert_H/Feb2023/MultiMaze_15stepped_gated_bar_noFood/Starved_noWater/230209/095735_s0a0_p0-0/Arena2/BallPositions_Processed.feather"
)

In [ ]:
reload["Time_sec"] = reload["Time"] / 80

In [ ]:
Curves = (
    hv.Curve(
        data=reload,
        kdims=["Time_sec"],
        vdims=[
            "ysmooth",
            "TrialNumber",
        ],
    )
    .groupby("TrialNumber")
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Viridis"),
        tools=[
            "hover",
        ],
        muted=True,
        xlabel="Time (s)",
        ylabel="Smoothed Y position (px)",
        fontscale=2,
    )
    .overlay()
)

Curves